# Spatial Lambda Fleming Viot process

Класс модели:
    - Аттрибуты:
        - Параметры модели
        - Вложенные классы:
            - Класс состояния:
                - Время
                - Список особей с их координатами в настоящий м
            

## Generate Poisson dynamic $\Pi$

In [251]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
import scipy.integrate as integr
from tqdm import tqdm
from icecream import ic

In [554]:
class State:
    def __init__(self, time=0):
        self.time = 0
        self.individuals=None
        self.hist=None
        self.freeId = 1
        
        
    def create(self, individuals):
        Ids = np.arange(self.freeId, self.freeId+len(individuals))
        self.individuals = np.column_stack((Ids, individuals))
        self.hist = np.column_stack((Ids, individuals))
        self.freeId = self.freeId+len(individuals)
        
        
    def delete(self, ids):
        self.individuals = np.delete(self.individuals, ids, axis=0)
    
    
    def add(self, individuals):
        Ids = np.arange(self.freeId, self.freeId+len(individuals))
        self.individuals = np.append(self.individuals, np.column_stack((Ids, individuals)),axis=0)
        self.hist = np.append(self.hist, np.column_stack((Ids, individuals)),axis=0)
        self.freeId = self.freeId+len(individuals)
        
    
    def genealogy(self, Id):
        lst = [[Id, self.hist[self.hist[:,0] == Id][0,2]]]
        parent_id = self.hist[self.hist[:,0] == Id][0,1]
        
        while parent_id!= 0:
            parent_time = self.hist[self.hist[:,0] == parent_id][0,2]
            lst.append([int(parent_id), parent_time])
            parent_id = self.hist[self.hist[:,0] == parent_id][0,1]
        return lst
    
    def build_tree(self, id1, id2):
        parents1, parents2 = self.genealogy(id1)[::-1], self.genealogy(id2)[::-1]
        # ic(parents1, parents2)
        for i in range(max(len(parents1), len(parents2))):
            if i < min(len(parents1), len(parents2)):
                if parents1[i] == parents2[i]:
                    print(f'{parents1[i][1]}\t{parents1[i][0]}')
                else:
                
                    print(f'{max(parents1[i][1],parents2[i][1])}\t{parents1[i][0]}\t{parents2[i][0]}')
            elif i < len(parents1):
                    print(f'{parents1[i][1]}\t{parents1[i][0]} \t ')
            else:
                print(f'{parents2[i][1]}\t\t {parents2[i][0]}')
        

In [620]:
class Model:
    def __init__(self, L=1, lamda=1, u0=0.4, rho=1000, theta=0.5, alpha=1, n_alleles=10):
        self.rho=rho
        self.L=L
        self.u0=u0
        self.theta=theta
        self.alpha=alpha
        self.lamda = lamda
        self.state = State()
        self.n_alleles = n_alleles
        self.dynamic=None
        
        
        
#--------------Initializating functions-------------------
        
        
    def generate_dynamic(self, n_epoch, time_init=0):
        '''Result is in (time, x, y)
        NOTE! MAYBE LAMBDA SHOULD BE RENORMALISED'''
        times = time_init + np.cumsum(np.random.exponential(self.lamda, n_epoch))
        xs = np.random.uniform(0, self.L, n_epoch)
        ys = np.random.uniform(0, self.L, n_epoch)
        self.dynamic = np.column_stack((xs, ys, times))
        
        
    def generate_initial_points(self):
        N_points = np.random.poisson(self.rho*self.L**2)
        xs = np.random.uniform(0, self.L, N_points)
        ys = np.random.uniform(0, self.L, N_points)
        return np.column_stack((xs, ys))
        
        
    def initiate(self, proport=0.5):
        N_points = np.random.poisson(self.rho*self.L**2)
        xs = np.random.uniform(0, self.L, N_points)
        ys = np.random.uniform(0, self.L, N_points)
        alleles = np.random.choice([0, 1], (N_points, self.n_alleles), p = [1 - proport, proport])
        pId = np.full(N_points, 0)
        times = np.full(N_points, 0)
        self.state.create(np.column_stack((pId, times, xs, ys ,alleles)))
        
    
    def choose_parent(self, z):
        probs = []
        for x in self.state.individuals:
            probs.append(self.v(z[0:2], x[3:5]))
        return self.state.individuals[np.random.choice(np.arange(len(self.state.individuals)), p = probs/np.sum(probs))]
    
    
    def choose_parent_type(self, z):
        probs = []
        for x in self.state.individuals:
            probs.append(self.v(z[0:2], x[3:5])) 
        return self.state.individuals[np.random.choice(np.arange(len(self.state.individuals)), p = probs/np.sum(probs))][5:]
    
    
    
#--------------Evolution functions--------------------------    
    def extinction(self, event):
        z, time = event[0:2], event[2]
        time = 0
        # time = event[2]
        ids = [] #ids to delete
        for i in range(len(self.state.individuals)):
            if np.random.uniform() < self.u(z, self.state.individuals[i,3:5]):
                ids.append(i)
                # print(f'{indicies=}')
        # print(f'survived {points[indicies].shape=}')
        self.state.delete(ids)
    
    
    def recolonization(self, event):
        z, time = event[0:2], event[2]
        parent = self.choose_parent(z)
        parentId = parent[0]
        parentType = parent[5:]
        intensity = lambda x, y: self.u(z, np.array([x,y]))
        max_intensity = intensity(z[0], z[1])
        total_intensity = integr.dblquad(intensity, 0, self.L, 0,  self.L )[0]
        # print(f"{total_intensity=}\n{max_intensity=}")
        n_points = np.random.poisson(self.rho * total_intensity) # Тут вроде total
        # print(f'{rho * total_intensity=}')
        # print(f'recolonized {n_points=}')
        points = []
        generated = 0
        while generated < n_points:
            x = np.random.uniform(0, self.L)
            y = np.random.uniform(0, self.L)

            if self.L**2 * intensity(x,y) >= np.random.uniform(0, max_intensity):
                points.append([x,y])
                generated += 1
        
        
        points = np.array(points,ndmin=2)
        points = np.column_stack(
            (
                np.full(n_points,parentId),
                np.full(n_points,time),
                points,
                np.full((n_points, self.n_alleles), parentType)
            )
        )
        self.state.add(points)
   

    
    
    
    def propagate(self, event):# parameters -- list [L, rho, u0, alpha, theta]
        # ic(event)
        # pass
        self.extinction(event)
        self.recolonization(event)
    
    
    def run(self):
        for event in tqdm(self.dynamic):
            self.propagate(event) 
    
#----------------Hat functions------------------------    
    def v(self, z, x):
        return np.exp(- np.linalg.norm(z-x)/(2 * self.alpha**2 * self.theta**2))
    
    
    def u(self, z, x):
        return self.u0 * np.exp(- np.linalg.norm(z-x)/(2 * self.theta**2))
    
    
    def h(self, z, x, beta = 1):
        return np.exp(-np.linalg.norm(z-x)/beta**2)
    
    
#---------------ANALYS FUNCTIONS----------------------- 
    def density(self, z, beta=1):
        points = self.state.individuals
        denom = 0
        thetas = np.zeros(self.n_alleles)
        for x in points:
            denom += self.h(z, x[3:5], beta)
            for k in range(self.n_alleles):
                if x[k+5] == 1:
                    thetas[k] += self.h(z, x[3:5], beta)
        thetas = thetas / denom

        return thetas


    def plt_SFS1(self, z, beta=1):
        d = self.density(z, beta=1)
        N = 100
        y = []
        for i in range(N):
            y.append((d<(i+1)/100).sum())
        plt.plot(y)


    def plt_SFS2(self, z1, z2,beta=1):
        d1 = self.density(z1, beta=1)
        d2 = self.density(z2, beta=1)
        N = 100
        y = np.zeros((N,N))
        for i in range(N):
            for j in range(N):
                y[i,j]=(np.logical_and(d1<(i+1)/N, d2<(j+1)/N).sum())
        plt.imshow(y, extent=[0,1,0,1])
        
        
    def plot_with_alleles(self, allele=0, alpha=0.5):
        points = self.state.individuals
        plt.scatter(points[points[:,5+allele]==0][:,3],points[points[:,5+allele]==0][:,4], alpha, label ='0 allele')
        plt.scatter(points[points[:,5+allele]==1][:,3],points[points[:,5+allele]==1][:,4], alpha, label = '1 allele')
        plt.legend()
        plt.show();
#--------------------Auxilary
    
    def save(self):
        return (self.rho, 
                self.L,
                self.u0,
                self.theta,
                self.alpha,
                self.lamda,
                self.n_alleles,
                np.copy(self.dynamic),
                np.copy(self.state.individuals),
                np.copy(self.state.hist))

    
    def load(self, data):
        self.rho, 
        self.L,
        self.u0,
        self.theta,
        self.alpha,
        self.lamda,
        self.n_alleles,
        self.dynamic,
        self.state.individuals,
        self.state.hist = data

In [621]:
a = Model(
    rho = 1000,
    L = 1,
    lamda = 1,
    u0 = 0.4,
    alpha = 1,
    theta = 0.3,
    n_alleles = 5
)

In [612]:
a.generate_dynamic(50)

In [613]:
a.initiate(0.4)

In [614]:
a.state.individuals.shape

(976, 10)

In [615]:
a.run()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:07<00:00,  7.10it/s]


In [616]:
saved = a.save()

In [617]:
b = Model(0,0,0,0,0,0,0)

In [618]:
b.load(saved)

In [619]:
b

In [565]:
a.state.genealogy(10800)

[[10800, 197.87072070869186],
 [9611, 164.04001956092856],
 [7678, 124.12829176537669],
 [7494, 122.18498108718764],
 [7446, 120.89947028461373],
 [6769, 108.3906892611479],
 [5931, 94.07883791569],
 [5860, 94.06171298416756],
 [5773, 92.9052448096736],
 [5032, 77.57182933468876],
 [772, 0.0]]

In [566]:
a.state.build_tree(10800, 10300)

0.0	772
77.57182933468876	5032
92.9052448096736	5773
94.06171298416756	5860
94.07883791569	5931
108.3906892611479	6769
120.89947028461373	7446
122.18498108718764	7494
124.12829176537669	7678
164.04001956092856	9611	9628
197.87072070869186	10800	10197
187.50386142875644		 10300


In [307]:
a.state.hist[a.state.hist[:,0]==3445]

array([], shape=(0, 10), dtype=float64)

In [516]:
a.state.hist[18000]

array([1.80010000e+04, 1.65430000e+04, 3.20570960e+02, 9.95288502e-01,
       8.05790258e-01, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       0.00000000e+00, 1.00000000e+00])

In [519]:
a.state.hist[a.state.hist[:,0]==16543]

array([[1.65430000e+04, 1.63580000e+04, 2.92674280e+02, 7.67969572e-01,
        5.35191458e-01, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 1.00000000e+00]])

In [520]:
a.state.hist[a.state.hist[:,0]==16538]

array([[1.65380000e+04, 1.63580000e+04, 2.92674280e+02, 8.71653873e-01,
        4.61146049e-01, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 1.00000000e+00]])

In [333]:
a.state.build_tree(20012, 20000)

ic| parents1: [0,
               2639,
               4414,
               5386,
               5823,
               5947,
               6107,
               7347,
               9829,
               11704,
               12176,
               12291,
               13089,
               13130,
               13396,
               13579,
               15726,
               16720,
               16912,
               19462,
               19620,
               20012]
    parents2: [0,
               2685,
               4556,
               6037,
               9044,
               9193,
               10532,
               12374,
               12841,
               12904,
               13787,
               14602,
               15228,
               15372,
               17187,
               18093,
               20000]


0
2639 	 2685
4414 	 4556
5386 	 6037
5823 	 9044
5947 	 9193
6107 	 10532
7347 	 12374
9829 	 12841
11704 	 12904
12176 	 13787
12291 	 14602
13089 	 15228
13130 	 15372
13396 	 17187
13579 	 18093
15726 	 20000
16720 	 
16912 	 
19462 	 
19620 	 
20012 	 


In [121]:
a.plt_SFS1(1, np.array([0.5,0.5]))